# Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
**The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. **

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with **any SQL database like Synapse** for example. ****The server should be created on the Resource Group where the Azure Cognitive Search service is located.
**
Let's begin..**

In [1]:
import os
import pandas as pd
import pyodbc

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase


from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [3]:
!sudo ./download_odbc_driver.sh

Downloading...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  17553      0 --:--:-- --:--:-- --:--:-- 17553
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0   2542      0 --:--:-- --:--:-- --:--:--  2542
Updating apt-get..
Get:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Ign:1 file:/var/nccl-repo-2.2.13-ga-cuda9.2  InRelease
Get:2 file:/var/nccl-repo-2.2.13-ga-cuda9.2  Release [574 B]                   
Get:2 file:/var/nccl-repo-2.2.13-ga-cuda9.2  Release [574 B]                   
Hit:3 http://azure.archive.ubuntu.com/ubuntu focal InRelease                   
Get:4 http://azure.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]  
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal In

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_NAME"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_NAME"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 17 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tJan  3 2024 16:32:18 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


/tmp/ipykernel_12433/2845560204.py:27: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = engine.execute("SELECT @@Version")


In [7]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    #Createse the table in Azure SQL
    engine.execute(create_table_sql)
    print("Table",table_name,"succesfully created")
    # Insert data into SQL Database
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

Table covidtracking succesfully created
rows: 0 - 1000 inserted
rows: 1000 - 2000 inserted
rows: 2000 - 3000 inserted
rows: 3000 - 4000 inserted
rows: 4000 - 5000 inserted
rows: 5000 - 6000 inserted
rows: 6000 - 7000 inserted
rows: 7000 - 8000 inserted
rows: 8000 - 9000 inserted
rows: 9000 - 10000 inserted
rows: 10000 - 11000 inserted
rows: 11000 - 12000 inserted
rows: 12000 - 13000 inserted
rows: 13000 - 14000 inserted
rows: 14000 - 15000 inserted
rows: 15000 - 16000 inserted
rows: 16000 - 17000 inserted
rows: 17000 - 18000 inserted
rows: 18000 - 19000 inserted
rows: 19000 - 20000 inserted
rows: 20000 - 20780 inserted


# Query with LLM

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right ENV variables and it will connect succesfully.

In [4]:
# Create or LLM Langchain object using GPT-4 deployment
# Again we need GPT-4. It is necesary in the use of Agents. GPT-35-Turbo will make many mistakes.
llm = AzureChatOpenAI(deployment_name="gpt-4-32k", temperature=0, max_tokens=500)#YK

In [5]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url) #initialize

In [6]:
# Natural Language question (query)
QUESTION = "How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states? Use the hospitalizedIncrease column"

### SQL Agent

Let's use an agent now and see how ReAct framework solves the problem.

In [8]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,# you provide tools to the agent
    top_k=30,
    verbose=True #shows intermediate steps
)

In [9]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.agent.allowed_tools

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

In [10]:
# And let's see now our clever crafted prompt
printmd(agent_executor.agent.llm_chain.prompt.template)



You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct mssql query to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to obtain, **ALWAYS** limit your query to at most 30 results.
- You can order the results by a relevant column to return the most interesting examples in the database.
- Never query for all the columns from a specific table, only ask for the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 
- Your response should be in Markdown. However, **when running  a SQL Query  in "Action Input", do not include the markdown backticks**. Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "Explanation:". Include the SQL query as part of the explanation section.
- If the question does not seem related to the database, just return "I don't know" as the answer.
- Only use the below tools. Only use the information returned by the below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the tools below.

## Tools:



sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
sql_db_list_tables: Input is an empty string, output is a comma separated list of tables in the database.
sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



## Use the following format:

Question: the input question you must answer. 
Thought: you should always think about what to do. 
Action: the action to take, should be one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]. 
Action Input: the input to the action. 
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. 
Final Answer: the final answer to the original input question. 

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: SELECT TOP (10) [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
Observation: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example



Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}

In [11]:
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: covidtracking
Thought:The `covidtracking` table seems to be the most relevant one for this question. I should check its schema to see what columns it has.
Action: sql_db_schema
Action Input: covidtracking
Observation: 
CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrease] INTEGER NULL, 
	[deathProbable] FLOAT(53) NULL, 
	hospitalized FLOAT(53) NULL, 
	[hospitalizedCumulative] FLOAT(53) NULL, 
	[hospitalizedCurrently] FLOAT(53) NULL, 
	[hospitalizedIncrease] INTEGER NULL, 
	[inIcuCumulative] FLOAT(53) NULL, 
	[inIcuCurrently] FLOAT(53) NULL, 
	negative FLOAT(53) NULL, 
	[negativeIncrease] INTEGER NULL, 
	[negativeTestsAntibody] FLOAT(53) NULL, 
	[negativeTestsPeopleAntibody] FLOAT(53) NULL, 
	[negativeTestsViral] FLO

In [15]:
printmd(response)

No patients were hospitalized in Texas during July 2020, and 63105 patients were hospitalized nationwide.

Explanation:
I queried the `covidtracking` table for the sum of the `hospitalizedIncrease` column where the date starts with '2020-07' and the state is 'TX' for Texas and for all states. The queries returned 0 and 63105 respectively. 
I used the following queries

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized
FROM covidtracking
WHERE date LIKE '2020-07%' AND state = 'TX'
```

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized
FROM covidtracking
WHERE date LIKE '2020-07%'
```

**IMPORTANT NOTE**: If you don't specify the column name on the question, runing the above cell multiple times will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we** achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.**

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to **create a custom REACT agent that connects to the internet using BING SEARCH API** to answer questions grounded on search results with citations. Basically a clone of Bing Chat.